In [3]:
%pip install scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/a2/6b/d96b05c50d84f8ab330f6615b021c9d348f1d783b9291cfc5500ada9de88/scikit_learn-1.3.1-cp39-cp39-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for scipy>=1.5.0 from https://files.pythonhosted.org/packages/3a/85/67af0d598ed6e60a105e7acbf6bb1be1eab8c2f22facd2ffec84ba9a431a/scipy-1.11.3-cp39-cp39-macosx_10_9_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for joblib>=1.1.1 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Obtaining dependency information for threadpoolctl>=2.0.0 from https://files.pythonhosted.org/packages/81/12/fd4dea011af9d69e1cad05c75f3f7202cdcbeac9b712eea58ca779a72865/threadpoolctl-3.2.0-py3-none

In [4]:
# Import necessary libraries
from datasets import load_dataset
from transformers import GPT2Tokenizer, GPT2Model
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, f1_score


# Task 1: Dataset Exploration
# Load the dbpedia_14 dataset
dataset = load_dataset('dbpedia_14')

# Quick exploration
print(dataset['train'].shape)
print(dataset['train'].features)
print(dataset['train'][0])



Generating test split: 100%|██████████| 70000/70000 [00:02<00:00, 25936.69 examples/s]

(560000, 3)
{'label': ClassLabel(names=['Company', 'EducationalInstitution', 'Artist', 'Athlete', 'OfficeHolder', 'MeanOfTransportation', 'Building', 'NaturalPlace', 'Village', 'Animal', 'Plant', 'Album', 'Film', 'WrittenWork'], id=None), 'title': Value(dtype='string', id=None), 'content': Value(dtype='string', id=None)}
{'label': 0, 'title': 'E. D. Abbott Ltd', 'content': ' Abbott of Farnham E D Abbott Limited was a British coachbuilding business based in Farnham Surrey trading under that name from 1929. A major part of their output was under sub-contract to motor vehicle manufacturers. Their business closed in 1972.'}


In [5]:
# Task 2: Data Pre-processing
# Tokenize the textual descriptions
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium')
tokenizer.pad_token = tokenizer.eos_token

def tokenize_and_format(examples):
    encodings = tokenizer(examples['content'], truncation=True, padding='max_length', max_length=256)
    encodings['labels'] = examples['label']
    return encodings

tokenized_datasets = dataset.map(tokenize_and_format, batched=True)
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

train_dataloader = DataLoader(tokenized_datasets['train'], shuffle=True, batch_size=8)
test_dataloader = DataLoader(tokenized_datasets['test'], batch_size=8)

Map: 100%|██████████| 70000/70000 [00:26<00:00, 2634.21 examples/s]


In [6]:
# Task 3: Model Building
class GPT2ForClassification(nn.Module):
    def __init__(self, num_labels=14):
        super(GPT2ForClassification, self).__init__()
        self.gpt2 = GPT2Model.from_pretrained('gpt2-medium')
        self.classifier = nn.Linear(self.gpt2.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.gpt2(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state
        logits = self.classifier(hidden_states[:, -1])
        return logits

model = GPT2ForClassification().to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))



In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# Task 4: Model Training
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
criterion = nn.CrossEntropyLoss()

num_epochs = 2  # Sample value. Can be increased as needed.
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        
        inputs, masks, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)
        
        logits = model(inputs, masks)
        loss = criterion(logits, labels)
        
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1} completed!")



In [ ]:
# Task 5: Model Evaluation
model.eval()

all_predictions = []
all_true_labels = []

with torch.no_grad():
    for batch in test_dataloader:
        inputs, masks, labels = batch['input_ids'], batch['attention_mask'], batch['labels']
        inputs, masks, labels = inputs.to(device), masks.to(device), labels.to(device)

        logits = model(inputs, masks)
        _, preds = torch.max(logits, dim=1)
        
        all_predictions.extend(preds.cpu().numpy())
        all_true_labels.extend(labels.cpu().numpy())

# Compute accuracy and F1 score
accuracy = accuracy_score(all_true_labels, all_predictions)
f1 = f1_score(all_true_labels, all_predictions, average='weighted')

print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")

